In [1]:
import torch
from torch.autograd import Variable
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms as tfs
from torchvision.datasets import MNIST


In [2]:
data_tf=tfs.Compose([
        tfs.ToTensor(),
        tfs.Normalize([0.5],[0.5]) # 标准化
        
        
    ])

train_set=MNIST("./data",train=True,transform=data_tf)
test_set=MNIST("./data",train=True,transform=data_tf)
train_data=DataLoader(train_set,64,True,num_workers=4)
test_data=DataLoader(test_set,128,False,num_workers=4)

In [3]:
# 定义模型
class rnn_classify(nn.Module):
    def __init__(self,in_feature=28,hidden_feature=100,num_class=10,num_layers=2):
        super(rnn_classify,self).__init__()
        self.rnn=nn.LSTM(in_feature,hidden_feature,num_layers) # 使用两层的LSTM
        # 将最后一个rnn 的输出使用全连接得到最后的分类结果
        self.classifier=nn.Linear(hidden_feature,num_class)
    
    def forward(self,x):
        # x的大小为(batch,1,28,28),所以需要将其转换为RNN的输入形式，即(28,batch,28)
        x=x.squeeze() # 去掉（batch,1,28,28 ）中的1 变成（batch,28,28）
        x=x.permute(2,0,1) # 将最后一维放到第一维，变成（28，batch，28）
        out,_=self.rnn(x) # 使用默认的隐藏状态，得到的out 是（28，batch,hidden_feature）
        out=out[-1,:,:] # 取序列中的最后一个，大小是（batch，hidden_feature）
        out=self.classifier(out) # 得到分类结果
        return out

In [4]:
net=rnn_classify()
criterion=nn.CrossEntropyLoss()
optimzier=torch.optim.Adadelta(net.parameters(),1e-1)

In [6]:
# 开始训练
from utils1 import train
train(net,train_data,test_data,10,optimzier,criterion)

IndexError: invalid index of a 0-dim tensor. Use tensor.item() to convert a 0-dim tensor to a Python number